# Oak parameter data from TRY

In [29]:
import pandas as pd
import os
import myfuncs
from datetime import datetime
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 400)

#define constants
m2_per_cm2 = 1e-4
g_biomass_per_g_C = 2
m2_per_mm2 = 1e-6
mg_per_g = 1e3
g_per_kg = 1000
mm2_per_cm2 = 100
g_per_mg = 1e-3
write_csv = True

In [27]:
path_to_trait_obs = "/home/adam/cloud/gdrive/postdoc/parameters/param_data/trait_observations"
write_QUKE_data_avail = False
write_to_csv = True

In [3]:
QUCH_try_data = os.path.join(path_to_trait_obs,"QUCH/TRY/TRY_obs_QUCH.txt")
QUKE_try_data = os.path.join(path_to_trait_obs,"QUKE/TRY/QUKE_TRYdata.txt")

### Functions

In [21]:
def convert_to_fates_units(trait_name,input_units,input_value):

    
    if ("SLA" in trait_name) | ("Specific leaf area" in trait_name) and input_units == "mm2 mg-1":  
        output_value = input_value * m2_per_mm2 * mg_per_g * g_biomass_per_g_C
        return output_value

    elif "Leaf nitrogen" in trait_name and input_units == "mg/g":
        output_value = input_value * g_per_mg * g_biomass_per_g_C
        return output_value

    elif "Stem specific density" in trait_name and input_units == "g/cm3":
        output_value = input_value
        return output_value
    
    # Anaker conversions
    elif trait_name == "log(SLA cm2 g-1)":
        output_value = np.exp(input_value) * m2_per_cm2 * g_biomass_per_g_C
        return output_value
        
    elif trait_name == 'log(wood density g ml-1)':
        output_value = np.exp(input_value)
        return output_value
        
    elif trait_name == 'log(N %)':
        output_value = np.exp(input_value) * 1e-2 * g_biomass_per_g_C
        return output_value
    
    #Jepson leaf N
    elif trait_name == "Leaf Nmass":
    
        # muliplying by 1e-2 is to reverse the "per cent"  
        output_value = input_value * 1e-2 * g_biomass_per_g_C
        return output_value
    
    elif "Wood density" in trait_name:
        output_value = input_value
        return output_value
    
    elif trait_name == "Leaf area to sapwood area" and input_units == "mm2 mm-2":
        output_value = input_value * 1e-4
        return output_value
    
    else:
        #print("No unit conversion known for:", trait_name)
        return None
    
def convert_to_fates_param_name(input_trait_name):
    if ("SLA" in input_trait_name) | ('Specific leaf area' in input_trait_name):
        return "fates_leaf_slatop"
    
    elif ("wood density" in input_trait_name) |  ("Wood density" in input_trait_name):
        return "fates_wood_density"
    
    elif ("N %" in input_trait_name) | ("Leaf Nmass" in input_trait_name):
        return "fates_stoich_nitr"
    
    elif "Leaf nitrogen (N) content per leaf dry mass" in input_trait_name:
        return "fates_stoich_nitr"
    
    elif "Leaf area to sapwood area" in input_trait_name:
        return "fates_allom_la_per_sa_int"
    
    else:
        return None
    
def get_sources(df,variable):
    return df.loc[df.fates_name == variable].source.unique()

def get_range_data(df,subset_name):

    summary_stats = df.groupby('fates_name')['fates_value'].agg(
        min_value='min',
        max_value='max',
        mean='mean',
        median='median',
        sample_size='size'
    )

    summary_stats['subset_name'] = subset_name
    
    return summary_stats

### View TRY data availability for black oak

Note: to view the data availability for QUKE look at the csv exported in this code block

In [13]:
#clean data
QUKE_data_avail = os.path.join(path_to_trait_obs,"QUKE/TRY/QUKE_TRY_obs_availability.txt")
QUKE_df = pd.read_csv(QUKE_data_avail, sep="\t", encoding='latin1')[["Trait","TraitID",".Quercus kelloggii."]]
QUKE_df = QUKE_df.rename(columns={"Trait": "trait", "TraitID": "trait_id", ".Quercus kelloggii.": "n"})
QUKE_df = QUKE_df.sort_values(["n"],ascending=False)

if write_QUKE_data_avail == True:
    QUKE_df.to_csv(os.path.join(path_to_trait_obs,"QUKE/TRY/QUKE_TRY_obs_availability_clean.csv"))

    #get trait IDs available for QUKE
    print(list(QUKE_df.trait_id))

    print(QUKE_df.head(20))

### Load, clean, and view oak trait data

In [14]:
# Load and join QUCH and QUKE data
try_df_raw_QUCH = pd.read_csv(QUCH_try_data, sep="\t", encoding='latin1')[['ObservationID','SpeciesName','AccSpeciesName','TraitID','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName','ErrorRisk']]
try_df_raw_QUKE = pd.read_csv(QUKE_try_data, sep="\t", encoding='latin1')[['ObservationID','SpeciesName','AccSpeciesName','TraitID','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName','ErrorRisk']]
try_df_raw = pd.concat([try_df_raw_QUCH,try_df_raw_QUKE])

try_df_raw = try_df_raw[['AccSpeciesName','TraitName','OrigValueStr','OrigUnitStr','StdValue','UnitName']]

print("Available trait observations")
print(try_df_raw.TraitName.value_counts())
print("\n")
print(try_df_raw.info())

Available trait observations
TraitName
Plant growth form                                                                                          76
Seed dry mass                                                                                              45
Leaf phenology type                                                                                        23
Plant height vegetative                                                                                    21
Plant lifespan (longevity)                                                                                 14
Plant human usage types                                                                                    13
Plant propagation type                                                                                      9
Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded     8
Plant woodiness                                                                  

### Filter to relevant traits

In [19]:
# Filter to get traits of interest
fates_relevant_traits = ['Stem specific density (SSD, stem dry mass per stem fresh volume) or wood density',
                         'Leaf nitrogen (N) content per leaf dry mass',
                         'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): undefined if petiole is in- or excluded',
                         'Leaf area per leaf dry mass (specific leaf area, SLA or 1/LMA): petiole excluded']

try_df = try_df_raw[try_df_raw.TraitName.isin(fates_relevant_traits)]
try_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 16 to 296
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AccSpeciesName  21 non-null     object 
 1   TraitName       21 non-null     object 
 2   OrigValueStr    21 non-null     object 
 3   OrigUnitStr     21 non-null     object 
 4   StdValue        21 non-null     float64
 5   UnitName        21 non-null     object 
dtypes: float64(1), object(5)
memory usage: 1.1+ KB


### Apply unit conversions and fates parameter names

In [20]:
# Unit convertion to fates units
fates_values = try_df.apply(lambda x: convert_to_fates_units(x['TraitName'], x['UnitName'], x["StdValue"]), axis=1)
try_df['fates_value'] = fates_values

# Convert variable names to fates names
fates_names = try_df.apply(lambda x: convert_to_fates_param_name(x["TraitName"]),axis = 1)
try_df["fates_name"] = fates_names


try_df

,AccSpeciesName,TraitName,OrigValueStr,OrigUnitStr,StdValue,UnitName,fates_value,fates_name
16,Quercus chrysolepis,"Stem specific density (SSD, stem dry mass per ...",0.7,g/cm^3,0.700000,g/cm3,0.700000,fates_wood_density
25,Quercus chrysolepis,Leaf nitrogen (N) content per leaf dry mass,1.04,%,10.400000,mg/g,0.020800,fates_stoich_nitr
71,Quercus chrysolepis,"Stem specific density (SSD, stem dry mass per ...",0.7,g/cm3,0.700000,g/cm3,0.700000,fates_wood_density
84,Quercus chrysolepis,Leaf area per leaf dry mass (specific leaf are...,155.861197443779,g m-2,6.415965,mm2 mg-1,0.012832,fates_leaf_slatop
15,Quercus kelloggii,Leaf area per leaf dry mass (specific leaf are...,117.73,g/m2,8.494012,mm2 mg-1,0.016988,fates_leaf_slatop
17,Quercus kelloggii,Leaf nitrogen (N) content per leaf dry mass,1.5765,%,15.765000,mg/g,0.031530,fates_stoich_nitr
18,Quercus kelloggii,Leaf area per leaf dry mass (specific leaf are...,84.9401172173617,cm2/g,8.494012,mm2 mg-1,0.016988,fates_leaf_slatop
44,Quercus kelloggii,Leaf area per leaf dry mass (specific leaf are...,111.89,cm2/g,11.189000,mm2 mg-1,0.022378,fates_leaf_slatop
46,Quercus kelloggii,"Stem specific density (SSD, stem dry mass per ...",0.72,g / cm3,0.720000,g/cm3,0.720000,fates_wood_density
47,Quercus kelloggii,Leaf nitrogen (N) content per leaf dry mass,1.82,%,18.200000,mg/g,0.036400,fates_stoich_nitr


In [26]:
QUKE_df = try_df.loc[try_df.AccSpeciesName == "Quercus kelloggii"]
QUCH_df = try_df.loc[try_df.AccSpeciesName == "Quercus chrysolepis"]

param_range_oak_df = pd.concat([get_range_data(QUKE_df,"QUKE"),get_range_data(QUCH_df,"QUCH")]).reset_index()
param_range_oak_df['sources'] = "try"
param_range_oak_df['pft'] = "oak"

In [30]:
# Write CSV
if write_to_csv == True:

    now = datetime.now()
    dt_string = now.strftime("%m-%d-%Y-%H-%M-%S")
    file_path = "oak_param_ranges_data_derived_" + dt_string + ".csv"
    param_range_oak_df.to_csv(file_path, index=False)

In [31]:
param_range_oak_df

,fates_name,min_value,max_value,mean,median,sample_size,subset_name,sources,pft
0,fates_leaf_slatop,0.016980,0.022378,0.018568,0.016988,8,QUKE,try,oak
1,fates_stoich_nitr,0.030600,0.039000,0.034243,0.033965,6,QUKE,try,oak
2,fates_wood_density,0.510000,0.720000,0.580000,0.510000,3,QUKE,try,oak
3,fates_leaf_slatop,0.012832,0.012832,0.012832,0.012832,1,QUCH,try,oak
4,fates_stoich_nitr,0.020800,0.020800,0.020800,0.020800,1,QUCH,try,oak
5,fates_wood_density,0.700000,0.700000,0.700000,0.700000,2,QUCH,try,oak
